In [1]:
%env BASE_DIR=/home/jupyter/prep

env: BASE_DIR=/home/jupyter/prep


In [2]:
#!gsutil cp gs://cs327e-open-access-2/shopify.zip $BASE_DIR

In [3]:
#!unzip ~/shopify.zip

In [4]:
import pandas as pd
from google.cloud import firestore

db = firestore.Client()
batch = db.batch()

#Read three files- apps, connector apps categories, and categories

df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
app = df.values.tolist()

df = pd.read_csv('shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
app_cat = df.values.tolist()

df = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n')
cat = df.values.tolist()

offset=0
count_apps = 0
for c in cat: #scanning each part of the categories COLLECTION
    ap_c = {}
    ap_c['id'] = c[0]
    ap_c['title'] = c[1]
    cat_ref = db.collection('categories').document(c[0])
    batch.set(cat_ref, ap_c)
    
    for ac in app_cat: #connector apps categories

        if c[0] == ac[1]:
            for a in app: #SCANNING APP SUBCOLLECTION
                if a[0] == ac[0]:
                    a_record = {}
                    a_record['id '] = a[0]
                    a_record['url'] = a[1]
                    a_record['title'] = a[2]
                    a_record['developer'] = a[3]
                    a_record['developer_link'] = a[4]
                    a_record['icon'] = a[5]
                    a_record['rating'] = a[6]
                    a_record['reviews_count'] = a[7]
                    apps_ref = cat_ref.collection('apps').document(a[0])
                    count_apps += 1 #count for apps subcollection
                    batch.set(apps_ref, a_record)
                #offset variable helps to commit in batches without server errors
                offset += 1
                if offset == 500:
                    batch.commit()
                    offset = 0
    
    batch.commit()
    print(f"category {c} has been loaded: TEST") #TESTING the loading as initial

category ['64430ad2835be8ad60c59e7d44e4b0b1', 'Customer support'] has been loaded: TEST
category ['737ad50051083aa051d127a53b3ac0da', 'Productivity'] has been loaded: TEST
category ['30a930262efca129caafcd586bc7e6fe', 'Places to sell'] has been loaded: TEST
category ['30ea2315e910c5e5335de42d5e87dca5', 'Orders and shipping'] has been loaded: TEST
category ['c769c2bd15500dd906102d9be97fdceb', 'Marketing'] has been loaded: TEST
category ['26a72de0d02e0e4e5f615332d61a878e', 'Finances'] has been loaded: TEST
category ['5eb4e29e50e3f178acc614236ed107f4', 'Sales and conversion optimization'] has been loaded: TEST
category ['e44cd5379a3166568741a07cc81ef6e5', 'Trust and security'] has been loaded: TEST
category ['c576a841fd4f333a6f074d68e76a1d37', 'Reporting'] has been loaded: TEST
category ['bd125b49ce6b0425d29e33cb8b49a496', 'Finding and adding products'] has been loaded: TEST
category ['9fe78acd078fc030de72c896adc93a2d', 'Inventory management'] has been loaded: TEST
category ['c3f9db73c29b

In [6]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()
#All of the apps collection and subcollections read
df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
app = df.values.tolist()

df = pd.read_csv('shopify/reviews.csv', sep=',', header=0, lineterminator='\n')
review = df.values.tolist()

df = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
pricing_plan = df.values.tolist()

df = pd.read_csv('shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
pricing_plan_feature = df.values.tolist()

df = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
key_benefit = df.values.tolist()

index=0
index1=0
count_pricing=0
for a in app: #app collection is scanned in
    ap = {}
    ap['id'] = a[0]
    ap['url'] = a[1]
    ap['title'] = a[2]
    ap['developer'] = a[3]
    ap['developer_link'] = a[4]
    ap['icon'] = a[5]
    ap['rating'] = a[6]
    ap['review_count'] = a[7]
    app_ref = db.collection('apps').document(a[0])
    batch.set(app_ref, ap)
    for r in review: #review subcollection scanned in
        if a[0] == r[0]:
            rev = {}
            rev['app_id'] = r[0]
            rev['author'] = r[1]
            rev['rating'] = r[2]
            rev['posted_at'] = r[3]
            review_ref = app_ref.collection('reviews').document(r[0]+str(index))
            batch.set(review_ref, rev)
            index+=1
    
    for p in pricing_plan:#pricing plan subcollection scanned in
        if a[0] == p[1]:
            pri = {}
            pri['id'] = p[0]
            pri['app_id'] = p[1]
            pri['title'] = p[2]
            pri['price'] = p[3]
            pricing_plan_ref = app_ref.collection('pricing_plans').document(p[0])
            batch.set(pricing_plan_ref, pri)
            
            for f in pricing_plan_feature:
                if p[0] == f[0]:
                    ft = {}
                    ft['feature'] = f[2]
                    features_ref = pricing_plan_ref.collection('pricing_plan_features').document(f[0])#PRICING PLAN HAS SUBCOLLECTION PRICING PLAN FEATURES SCANNED IN
                    count_pricing += 1 #COUNT VARIABLE FOR pricing plan features
                    batch.set(features_ref, ft)                
    for k in key_benefit: #key benefits subcollection scanned in
        if a[0] == k[0]:
            key = {}
            key['app_id'] = k[0]
            key['title'] = k[1]
            key['description'] = k[2]
            key_benefit_ref = app_ref.collection('key_benefits').document(k[0]+str(index1))
            batch.set(key_benefit_ref, key)
            index1+=1

    batch.commit()

In [7]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter
from google.cloud.firestore_v1 import aggregation
#reading documents on each one
db = firestore.Client()
query = db.collection('apps').where(filter=FieldFilter('id', '!=', '')) 
results = aggregation.AggregationQuery(query).count().get()
    
for result in results:
    print(f"Count of Apps in the collection: {result[0].value}")

Count of Apps in the collection: 3547


In [8]:
count = 0
apps_collection = db.collection('apps')
for i in apps_collection.stream():
    general = apps_collection.document(i.id)
    query = general.collection('reviews').where(filter=FieldFilter('app_id', '!=', '')) 
    results = aggregation.AggregationQuery(query).count().get()
    for result in results:
        count += result[0].value
print(f"Count of documents in Reviews:{count}")

Count of documents in Reviews:124601.0


In [9]:
count = 0 
for i in apps_collection.stream():
    general = apps_collection.document(i.id)
    query = general.collection('key_benefits').where(filter=FieldFilter('app_id', '!=', '')) 
    results = aggregation.AggregationQuery(query).count().get()
    for result in results:
        count += result[0].value
print(f"Count of documents in Key Benefits:{count}")

count = 0 
for i in apps_collection.stream():
    general = apps_collection.document(i.id)
    query = general.collection('pricing_plans').where(filter=FieldFilter('id', '!=', '')) 
    results = aggregation.AggregationQuery(query).count().get()
    for result in results:
        count += result[0].value
print(f"Count of documents in Pricing Plans:{count}")

Count of documents in Key Benefits:9541.0
Count of documents in Pricing Plans:6275


In [10]:
print(f"Count of documents in Pricing Plan Features:{count_pricing}")

Count of documents in Pricing Plan Features:16270


In [11]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter
from google.cloud.firestore_v1 import aggregation

db = firestore.Client()
query = db.collection('categories').where(filter=FieldFilter('id', '!=', '')) 
results = aggregation.AggregationQuery(query).count().get() 
for result in results:
    print(f"Category Counts in Categories collection: {result[0].value}")

Category Counts in Categories collection: 12


In [12]:
print(f"Count of documents in Apps Subcollection: {count_apps}")

Count of documents in Apps Subcollection: 5383


Query 1 for Top 10 Apps in Productivity

In [13]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter

In [14]:
db = firestore.Client()
query = db.collection('categories').where(filter=FieldFilter('title', '==', 'Productivity')).stream()
for result in query:
    query2 = db.collection('categories').document(result.id).collection('apps').order_by("rating", direction=firestore.Query.DESCENDING).limit(10).stream()
    for result in query2:
        master = result.to_dict()
        print(f'''App ID: {result.id}
Title: {master["title"]}
Developer: {master["developer"]}
Rating: {master["rating"]}
Reviews Count: {master["reviews_count"]}''')
        print()

App ID: fe5ae45c-379c-42bf-be7d-b5f6e15fc13c
Title: Customer Tags
Developer: Union Works Apps
Rating: 5.0
Reviews Count: 21

App ID: f99bb1e3-f326-4f10-8901-491652e9809b
Title: Order Tagger
Developer: Union Works Apps
Rating: 5.0
Reviews Count: 68

App ID: f864e3bd-da0e-41dc-be65-984325331475
Title: SilkRoad ‑ Facebook Auto Ads
Developer: SilkRoad
Rating: 5.0
Reviews Count: 2

App ID: f5344e64-9cda-4d97-b198-2aaeb5170518
Title: Xporter Data Export Tool
Developer: Modd Apps Inc.
Rating: 5.0
Reviews Count: 223

App ID: ed77a32d-0fa3-458b-b639-e01ea7b78ec0
Title: FraudBlock Fraud Prevention
Developer: ShopFox
Rating: 5.0
Reviews Count: 6

App ID: e0e231d6-4988-4a8c-ad84-cf18d0f38738
Title: Automation Fox
Developer: Automation Fox
Rating: 5.0
Reviews Count: 1

App ID: d75fa395-3a4d-41c3-88d1-2f802c1e2411
Title: AirPower
Developer: BaseGenius
Rating: 5.0
Reviews Count: 1

App ID: d0be03ff-f74d-4fb5-8d2c-79541a1aea1b
Title: Excelify
Developer: Excelify.io
Rating: 5.0
Reviews Count: 178

App 

Query 2 for Top 10 Apps Review Count 

In [15]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter

In [16]:
db = firestore.Client()
query = db.collection('apps').order_by('review_count', direction=firestore.Query.DESCENDING).limit(10).stream()
for res in query:
        master = res.to_dict()
        print(f'''App ID: {res.id}
Title: {master["title"]}
Developer: {master["developer"]}
Rating: {master["rating"]}
Reviews Count: {master["review_count"]}''')
        print()
    

App ID: d9f142ee-b141-4dc4-9353-173db61d2eb0
Title: Privy ‑ Exit Pop Ups & Email
Developer: Privy
Rating: 4.7
Reviews Count: 23078

App ID: 78ea0810-c008-4a4e-a82f-de0c790e3286
Title: Free Shipping Bar
Developer: Hextom
Rating: 4.9
Reviews Count: 8737

App ID: b88488b0-9912-44d3-b736-224c36f09d95
Title: Sales Pop ‑ Popup Notification
Developer: CartKit
Rating: 4.8
Reviews Count: 6905

App ID: e528a60e-94f8-4e92-80e2-5bc6013b8283
Title: BEST Currency Converter
Developer: Grizzly Apps
Rating: 4.8
Reviews Count: 5986

App ID: be2640c4-01b5-4d52-9f68-cae8c0734d0d
Title: Recart FB Messenger Marketing
Developer: Recart
Rating: 4.8
Reviews Count: 5596

App ID: 70bff9e0-4316-4cc6-84ce-92fcd1bc6925
Title: EU Cookie Bar ‑ Cookie GDPR
Developer: Booster Apps
Rating: 4.7
Reviews Count: 5259

App ID: 171816e2-27d4-4552-a65e-ab44a312fe04
Title: Sales Pop Master ‑ Countdown
Developer: Autoketing
Rating: 4.8
Reviews Count: 4931

App ID: 9025eff0-d714-4df1-930f-43f5582979ad
Title: Ultimate Sales Boost
